#### In this project, i will examine if there has been a change in temperature in the EU over the past two decades and if there are factors that might have had an impact on those changes (if they exist).

# Temperature Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# global average monthly temperatures from 1743 to 2013
temp_country = pd.read_csv('GlobalLandTemperatures_GlobalLandTemperaturesByCountry.csv')
temp_country.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
Country                           object
dtype: object

In [3]:
# dt (date) not in datetime format, changing that
temp_country['dt'] = pd.to_datetime(temp_country['dt'])
temp_country.dtypes

dt                               datetime64[ns]
AverageTemperature                      float64
AverageTemperatureUncertainty           float64
Country                                  object
dtype: object

In [4]:
# select EU countries
eu_temp_country = temp_country[temp_country.Country.str.contains('Austria|Belgium|Bulgaria|Croatia|Cyprus|Czech Republic|Denmark|Estonia|Finland|France|Germany|Greece|Hungary|Ireland|Italy|Latvia|Lithuania|Luxembourg|Malta|Netherlands|Poland|Portugal|Romania|Slovakia|Slovenia|Spain|Sweden')]
# France, Denmark Netherlands were in there twice (one of the times with '(europe)' at the end)
eu_temp_country = eu_temp_country[~eu_temp_country.Country.str.contains('Europe')]
# # keeping only data from 2000 onwards (September of 2013 was NaN for every country, so deleting it as well)
# eu_temp_country = eu_temp_country[~eu_temp_country.dt.str.contains('17|18|19|2013-09-01')]
eu_temp_country = eu_temp_country[(eu_temp_country['dt'] > '1969-12-01') & (eu_temp_country['dt'] < '2013-09-01')]
# dropping the uncertainty column as i dont need it for my analysis
eu_temp_country = eu_temp_country.drop(['AverageTemperatureUncertainty'], axis = 1)
eu_temp_country

,dt,AverageTemperature,Country
39465,1970-01-01,-3.892,Austria
39466,1970-02-01,-3.116,Austria
39467,1970-03-01,-0.590,Austria
39468,1970-04-01,4.328,Austria
39469,1970-05-01,9.018,Austria
...,...,...,...
508629,2013-04-01,1.059,Sweden
508630,2013-05-01,10.509,Sweden
508631,2013-06-01,13.741,Sweden
508632,2013-07-01,15.262,Sweden


In [5]:
# checking dtypes
eu_temp_country.dtypes
# # dt (date) not in datetime format, changing that
# eu_temp_country['dt'] = pd.to_datetime(eu_temp_country['dt'])

dt                    datetime64[ns]
AverageTemperature           float64
Country                       object
dtype: object

In [6]:
# renaming dt to date, standardizing column names
eu_temp_country = eu_temp_country.rename(columns={"dt": "date"})
eu_temp_country.columns = [x.lower() for x in eu_temp_country.columns]
# eu_temp_country.columns = eu_temp_country.columns.str.replace(" ", "_", regex=True)
# resetting index
eu_temp_country.reset_index(inplace=True)
eu_temp_country = eu_temp_country.drop(['index'], axis = 1)
eu_temp_country

,date,averagetemperature,country
0,1970-01-01,-3.892,Austria
1,1970-02-01,-3.116,Austria
2,1970-03-01,-0.590,Austria
3,1970-04-01,4.328,Austria
4,1970-05-01,9.018,Austria
...,...,...,...
14143,2013-04-01,1.059,Sweden
14144,2013-05-01,10.509,Sweden
14145,2013-06-01,13.741,Sweden
14146,2013-07-01,15.262,Sweden


In [7]:
# checking for NaNs
eu_temp_country.isnull().sum()

date                  0
averagetemperature    0
country               0
dtype: int64

In [8]:
# saving the Dataframe 
eu_temp_country.to_csv('EU_Countries_2000_2013_AVG_Temp.csv', index=False)

In [9]:
eu_temp_country

,date,averagetemperature,country
0,1970-01-01,-3.892,Austria
1,1970-02-01,-3.116,Austria
2,1970-03-01,-0.590,Austria
3,1970-04-01,4.328,Austria
4,1970-05-01,9.018,Austria
...,...,...,...
14143,2013-04-01,1.059,Sweden
14144,2013-05-01,10.509,Sweden
14145,2013-06-01,13.741,Sweden
14146,2013-07-01,15.262,Sweden


### Cleaned the first dataframe, now the next one.

In [10]:
# Table with temperature changes
eu_temp_change_country = pd.read_csv('FAOSTAT_data_en_3-21-2023 (1).csv')
eu_temp_change_country

,Domain,Area,Element,Months,Year,Value
0,Temperature change,Austria,Temperature change,January,1970,0.049
1,Temperature change,Austria,Temperature change,January,1971,-0.351
2,Temperature change,Austria,Temperature change,January,1972,-0.412
3,Temperature change,Austria,Temperature change,January,1973,0.739
4,Temperature change,Austria,Temperature change,January,1974,3.377
...,...,...,...,...,...,...
12019,Temperature change,Sweden,Temperature change,December,2009,-0.984
12020,Temperature change,Sweden,Temperature change,December,2010,-5.641
12021,Temperature change,Sweden,Temperature change,December,2011,4.159
12022,Temperature change,Sweden,Temperature change,December,2012,-2.813


In [11]:
# changing months to numbers (applying the first of each month to adjust to eu_temp_country)
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('January', '-01-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('February', '-02-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('March', '-03-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('April', '-04-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('May', '-05-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('June', '-06-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('July', '-07-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('August', '-08-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('September', '-09-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('October', '-10-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('November', '-11-01')
eu_temp_change_country.Months = eu_temp_change_country.Months.str.replace('December', '-12-01')


In [12]:
# concatinating Months and Year column
eu_temp_change_country["date"] = eu_temp_change_country['Year'].astype(str) +""+ eu_temp_change_country["Months"]


# sorting by date
eu_temp_change_country.sort_values(by=['Area','date'], inplace = True) 
eu_temp_change_country

,Domain,Area,Element,Months,Year,Value,date
0,Temperature change,Austria,Temperature change,-01-01,1970,0.049,1970-01-01
44,Temperature change,Austria,Temperature change,-02-01,1970,-0.826,1970-02-01
88,Temperature change,Austria,Temperature change,-03-01,1970,-1.925,1970-03-01
132,Temperature change,Austria,Temperature change,-04-01,1970,-1.409,1970-04-01
176,Temperature change,Austria,Temperature change,-05-01,1970,-1.559,1970-05-01
...,...,...,...,...,...,...,...
11847,Temperature change,Sweden,Temperature change,-08-01,2013,1.221,2013-08-01
11891,Temperature change,Sweden,Temperature change,-09-01,2013,1.588,2013-09-01
11935,Temperature change,Sweden,Temperature change,-10-01,2013,1.318,2013-10-01
11979,Temperature change,Sweden,Temperature change,-11-01,2013,1.869,2013-11-01


In [13]:
# dropping row sept - dec 2013 to make it equal to eu_temp_country
eu_temp_change_country = eu_temp_change_country[~eu_temp_change_country.date.str.contains('2013-09-01|2013-10-01|2013-11-01|2013-12-01')]
eu_temp_change_country

,Domain,Area,Element,Months,Year,Value,date
0,Temperature change,Austria,Temperature change,-01-01,1970,0.049,1970-01-01
44,Temperature change,Austria,Temperature change,-02-01,1970,-0.826,1970-02-01
88,Temperature change,Austria,Temperature change,-03-01,1970,-1.925,1970-03-01
132,Temperature change,Austria,Temperature change,-04-01,1970,-1.409,1970-04-01
176,Temperature change,Austria,Temperature change,-05-01,1970,-1.559,1970-05-01
...,...,...,...,...,...,...,...
11671,Temperature change,Sweden,Temperature change,-04-01,2013,0.399,2013-04-01
11715,Temperature change,Sweden,Temperature change,-05-01,2013,3.458,2013-05-01
11759,Temperature change,Sweden,Temperature change,-06-01,2013,1.420,2013-06-01
11803,Temperature change,Sweden,Temperature change,-07-01,2013,1.009,2013-07-01


In [14]:
# changing date column into datetime format
eu_temp_change_country['date'] = pd.to_datetime(eu_temp_change_country['date'])

C:\Users\Customer\AppData\Local\Temp\ipykernel_24212\1832717746.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eu_temp_change_country['date'] = pd.to_datetime(eu_temp_change_country['date'])


In [15]:
# sorting by date
eu_temp_change_country.sort_values(by=['Area','date'], inplace = True) 
eu_temp_change_country

C:\Users\Customer\AppData\Local\Temp\ipykernel_24212\895701952.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eu_temp_change_country.sort_values(by=['Area','date'], inplace = True)


,Domain,Area,Element,Months,Year,Value,date
0,Temperature change,Austria,Temperature change,-01-01,1970,0.049,1970-01-01
44,Temperature change,Austria,Temperature change,-02-01,1970,-0.826,1970-02-01
88,Temperature change,Austria,Temperature change,-03-01,1970,-1.925,1970-03-01
132,Temperature change,Austria,Temperature change,-04-01,1970,-1.409,1970-04-01
176,Temperature change,Austria,Temperature change,-05-01,1970,-1.559,1970-05-01
...,...,...,...,...,...,...,...
11671,Temperature change,Sweden,Temperature change,-04-01,2013,0.399,2013-04-01
11715,Temperature change,Sweden,Temperature change,-05-01,2013,3.458,2013-05-01
11759,Temperature change,Sweden,Temperature change,-06-01,2013,1.420,2013-06-01
11803,Temperature change,Sweden,Temperature change,-07-01,2013,1.009,2013-07-01


In [16]:
# checkin dtypes to see if date conversion worked
eu_temp_change_country.dtypes

Domain             object
Area               object
Element            object
Months             object
Year                int64
Value             float64
date       datetime64[ns]
dtype: object

In [17]:
# dropping Element, Months, Year, Domain Unit Column
eu_temp_change_country = eu_temp_change_country.drop(['Domain','Element', 'Months', 'Year'], axis = 1)

In [18]:
# resetting index
eu_temp_change_country.reset_index(inplace=True)
eu_temp_change_country = eu_temp_change_country.drop(['index'], axis = 1)
eu_temp_change_country

,Area,Value,date
0,Austria,0.049,1970-01-01
1,Austria,-0.826,1970-02-01
2,Austria,-1.925,1970-03-01
3,Austria,-1.409,1970-04-01
4,Austria,-1.559,1970-05-01
...,...,...,...
11911,Sweden,0.399,2013-04-01
11912,Sweden,3.458,2013-05-01
11913,Sweden,1.420,2013-06-01
11914,Sweden,1.009,2013-07-01


In [19]:
# renaming Area to Country and Value to Temperature change
eu_temp_change_country = eu_temp_change_country.rename(columns={"Area" : "country", "Value" : "temperature_change"})
eu_temp_change_country

,country,temperature_change,date
0,Austria,0.049,1970-01-01
1,Austria,-0.826,1970-02-01
2,Austria,-1.925,1970-03-01
3,Austria,-1.409,1970-04-01
4,Austria,-1.559,1970-05-01
...,...,...,...
11911,Sweden,0.399,2013-04-01
11912,Sweden,3.458,2013-05-01
11913,Sweden,1.420,2013-06-01
11914,Sweden,1.009,2013-07-01


### Cleaned the second DataFrame, now merging them together.

In [20]:
# merging eu_temp_change_country with eu_temp_country
eu_temp_change_avg = pd.merge(left = eu_temp_country, 
                              right = eu_temp_change_country,
                              how = 'left', 
                              left_on = ['country', 'date'], 
                              right_on = ['country', 'date'])
eu_temp_change_avg

,date,averagetemperature,country,temperature_change
0,1970-01-01,-3.892,Austria,0.049
1,1970-02-01,-3.116,Austria,-0.826
2,1970-03-01,-0.590,Austria,-1.925
3,1970-04-01,4.328,Austria,-1.409
4,1970-05-01,9.018,Austria,-1.559
...,...,...,...,...
14143,2013-04-01,1.059,Sweden,0.399
14144,2013-05-01,10.509,Sweden,3.458
14145,2013-06-01,13.741,Sweden,1.420
14146,2013-07-01,15.262,Sweden,1.009


In [21]:
# checking if merging went right
display(eu_temp_change_avg.head(5))
display(eu_temp_country.head(5))
display(eu_temp_change_country.head(5))

,date,averagetemperature,country,temperature_change
0,1970-01-01,-3.892,Austria,0.049
1,1970-02-01,-3.116,Austria,-0.826
2,1970-03-01,-0.590,Austria,-1.925
3,1970-04-01,4.328,Austria,-1.409
4,1970-05-01,9.018,Austria,-1.559


,date,averagetemperature,country
0,1970-01-01,-3.892,Austria
1,1970-02-01,-3.116,Austria
2,1970-03-01,-0.590,Austria
3,1970-04-01,4.328,Austria
4,1970-05-01,9.018,Austria


,country,temperature_change,date
0,Austria,0.049,1970-01-01
1,Austria,-0.826,1970-02-01
2,Austria,-1.925,1970-03-01
3,Austria,-1.409,1970-04-01
4,Austria,-1.559,1970-05-01


In [22]:
# reordering columns for readability
eu_temp_change_avg = eu_temp_change_avg[['country', 'date', 'averagetemperature', 'temperature_change']]
eu_temp_change_avg

,country,date,averagetemperature,temperature_change
0,Austria,1970-01-01,-3.892,0.049
1,Austria,1970-02-01,-3.116,-0.826
2,Austria,1970-03-01,-0.590,-1.925
3,Austria,1970-04-01,4.328,-1.409
4,Austria,1970-05-01,9.018,-1.559
...,...,...,...,...
14143,Sweden,2013-04-01,1.059,0.399
14144,Sweden,2013-05-01,10.509,3.458
14145,Sweden,2013-06-01,13.741,1.420
14146,Sweden,2013-07-01,15.262,1.009


In [ ]:
# eu_temp_change_avg[eu_temp_change_avg.country.str.contains('Denmark')]
# # somehow, denmarks values are complety ridiculus, will have to exclude them 

### Still some NaNs left, dealing with them now.

In [23]:
# checking for NaNs
eu_temp_change_avg.isnull().sum()

country                  0
date                     0
averagetemperature       0
temperature_change    2840
dtype: int64

In [24]:
# splitting off all rows with NaNs into seperate DataFrame and to do a value count
null_table = eu_temp_change_avg[eu_temp_change_avg.isna().any(axis=1)]
null_table.country.value_counts()

Czech Republic    524
Belgium           360
Luxembourg        360
Slovakia          276
Croatia           264
Estonia           264
Latvia            264
Lithuania         264
Slovenia          264
Name: country, dtype: int64

In [25]:
# filling all NaNs with the overall mean
eu_temp_change_avg.temperature_change = eu_temp_change_avg['temperature_change'].fillna(eu_temp_change_avg['temperature_change'].mean())
eu_temp_change_avg.isnull().sum()

country               0
date                  0
averagetemperature    0
temperature_change    0
dtype: int64

##### I started seperating the countries to fill their NaNs with the mean value of there country specific temp_changes. But the i saw, that the means only start to differ from the third decimal points on (so in the 1/1000).

##### As this difference is so insignificant, i decided to fill the NaNs with the overall mean, as this will be good enough for my analysis

##### In the 3 cells below, you can see an example of what i mean.

In [26]:
# slovenia = eu_temp_change_avg[eu_temp_change_avg.country.str.contains('Slovenia')]
# slovenia.temperature_change = slovenia['temperature_change'].fillna(slovenia['temperature_change'].mean())
# slovenia

In [27]:
# Lithuania = eu_temp_change_avg[eu_temp_change_avg.country.str.contains('Lithuania')]
# Lithuania.temperature_change = Lithuania['temperature_change'].fillna(Lithuania['temperature_change'].mean())
# Lithuania

In [28]:
# Estonia = eu_temp_change_avg[eu_temp_change_avg.country.str.contains('Estonia')]
# Estonia.temperature_change = Estonia['temperature_change'].fillna(Estonia['temperature_change'].mean())
# Estonia

In [29]:
# saving the final temperature table
eu_temp_change_avg.to_csv('Final_Temperature_Table.csv', index=False)

### Temperature Table cleaned, merged and saved to csv.

# Cars per 1000 inhabitants data:

In [150]:
cars = pd.read_csv('cars_per_1000_inhabitants_2.csv', sep=';')
cars

,TIME,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Belgium,387,396,399,407,416,421,427,433,440,448,456,460,462,464,467,468,470,473,477,479,480,488,489,491,494,497,503,508,511,511,510,510
1,Bulgaria,152,158,166,178,188,197,205,209,220,233,245,265,279,298,317,333,233,277,317,337,353,368,385,402,418,442,443,393,396,407,414,414
2,Czechia,234,247.1,260.1,274,283,295,310,329,339,335,336,346,358,364,374,387,401,414,424,424,429,436,448,450,459,485,502,522,540,554,565,579
3,Denmark,309,309,310,311,309,320,330,337,342,346,347,349,351,455,455.1,455.1,455,466,468,442.1,381,394,399,405,412,419,429,438,447,455,466,475
4,Germany,385,390,433,477,488,494,500,504,508,515,532,538,541,546,550,559,566,501,504,510,527,534,539,543,547,548,555,561,567,574,580,583
5,Estonia,154,171,188,215,233,269,289,307,327,327,333,294,291,318,347,366,413,391,413,409,416,433,456,478,497,514,534,550,563,598,608,620
6,Ireland,227,236,241,249,261,274,289,307,321,336,344,360,371,380,385,395,410,422,442,434,426,428,426,427,431,436,439,444,445,454,458,456
7,Greece,169,171,175,187,197,208,220,234,249,272,295,314,334,351,371,391,412,434,453,462,469,469,470,469,471,474,479,487,493,504,514,529
8,Spain,309,321,334,341,346,357,371.1,385.1,400.1,416,429,442,448,439,451,460.1,470,476,479,473,475,476,476,474,474,481,492,504,513,519,521,525
9,France,404,406,408,413,420,422,427,435,446,454,460,467,471,475,476,476,478,480,483.1,486,538,540,542,546,551,558,564,569,569,571,569,571


##### Some fields have ' : '  in them instead of NaNs. There were also fields with the ' : ' within the columns and rows i am going to keep. I dealt with those in excel because that was faster. To keep track of the fields, where i put a value in, i added a ' .1 ' to them. The values i added are always the mean change between the last and next neighbors.

In [151]:
# dropping rows 39 - 45 (because all NaNs)
cars = cars[cars['TIME'].isna() == False]
#cars

In [152]:
# renaming 'TIME' column to 'country'
cars = cars.rename(columns={"TIME" : "country"})
cars = cars.sort_values('country')
#cars

In [153]:
# value count on Temp Table to see which countries to keep in cars
eu_temp_change_avg.country.value_counts()

Austria           524
Italy             524
Spain             524
Slovenia          524
Slovakia          524
Romania           524
Portugal          524
Poland            524
Netherlands       524
Malta             524
Luxembourg        524
Lithuania         524
Latvia            524
Ireland           524
Belgium           524
Hungary           524
Greece            524
Germany           524
France            524
Finland           524
Estonia           524
Denmark           524
Czech Republic    524
Cyprus            524
Croatia           524
Bulgaria          524
Sweden            524
Name: country, dtype: int64

In [154]:
# selecting the right countries
cars = cars[~cars.country.str.contains('Iceland|United Kingdom|Türkiye|Switzerland|Serbia|Norway|North Macedonia|Montenegro|Liechtenstein|Kosovo|Bosnia and Herzegovina|Albania')]
# resetting index
cars.reset_index(inplace=True)
cars = cars.drop(['index'], axis = 1)
# cars

In [155]:
# renaming countries to how they are nemd in temperature table
# germany : deleting the (...) and czechia to czech republic
cars.country = cars.country.str.replace('Czechia', 'Czech Republic')
cars

,country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Austria,388,397,412,425,438,452,463,475,487,501,511,519,492,498,501,504,508,511,514,522,530,537,542,546,547,546,550,555,562,566,570,572
1,Belgium,387,396,399,407,416,421,427,433,440,448,456,460,462,464,467,468,470,473,477,479,480,488,489,491,494,497,503,508,511,511,510,510
2,Bulgaria,152,158,166,178,188,197,205,209,220,233,245,265,279,298,317,333,233,277,317,337,353,368,385,402,418,442,443,393,396,407,414,414
3,Croatia,166,160,147,139,150,155,184,205,221,235,261,275,285,298,308,319,331,350,360,358,355,355,339,341,349,358,374,389,409,425,433,463
4,Cyprus,305,315,320,321,325,335,341,348,365,372,384,397,404,419,458,477,492,529,557,563,551,545,549,553,565,575,595,609,629,645,645,655
5,Czech Republic,234,247.1,260.1,274,283,295,310,329,339,335,336,346,358,364,374,387,401,414,424,424,429,436,448,450,459,485,502,522,540,554,565,579
6,Denmark,309,309,310,311,309,320,330,337,342,346,347,349,351,455,455.1,455.1,455,466,468,442.1,381,394,399,405,412,419,429,438,447,455,466,475
7,Estonia,154,171,188,215,233,269,289,307,327,327,333,294,291,318,347,366,413,391,413,409,416,433,456,478,497,514,534,550,563,598,608,620
8,Finland,388,382,383,369,367,372,379,378,392,403,412,416,422,436,448,462,475,485,507,519,535,551,560,570,580,590,604,617,629,642,652,656
9,France,404,406,408,413,420,422,427,435,446,454,460,467,471,475,476,476,478,480,483.1,486,538,540,542,546,551,558,564,569,569,571,569,571


In [156]:
# dropping year columns 2014 - 2021
cars = cars.drop(['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'], axis = 1)

In [157]:
# force to show all columns
pd.set_option('display.max_columns', None)
cars

,country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Austria,388,397,412,425,438,452,463,475,487,501,511,519,492,498,501,504,508,511,514,522,530,537,542,546
1,Belgium,387,396,399,407,416,421,427,433,440,448,456,460,462,464,467,468,470,473,477,479,480,488,489,491
2,Bulgaria,152,158,166,178,188,197,205,209,220,233,245,265,279,298,317,333,233,277,317,337,353,368,385,402
3,Croatia,166,160,147,139,150,155,184,205,221,235,261,275,285,298,308,319,331,350,360,358,355,355,339,341
4,Cyprus,305,315,320,321,325,335,341,348,365,372,384,397,404,419,458,477,492,529,557,563,551,545,549,553
5,Czech Republic,234,247.1,260.1,274,283,295,310,329,339,335,336,346,358,364,374,387,401,414,424,424,429,436,448,450
6,Denmark,309,309,310,311,309,320,330,337,342,346,347,349,351,455,455.1,455.1,455,466,468,442.1,381,394,399,405
7,Estonia,154,171,188,215,233,269,289,307,327,327,333,294,291,318,347,366,413,391,413,409,416,433,456,478
8,Finland,388,382,383,369,367,372,379,378,392,403,412,416,422,436,448,462,475,485,507,519,535,551,560,570
9,France,404,406,408,413,420,422,427,435,446,454,460,467,471,475,476,476,478,480,483.1,486,538,540,542,546


In [160]:
cars.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013'],
      dtype='object')>

In [162]:
cars[['1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']] = cars[['1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']].astype(np.number)
print(cars.dtypes)
cars

country     object
1990       float64
1991       float64
1992       float64
1993       float64
1994       float64
1995       float64
1996       float64
1997       float64
1998       float64
1999       float64
2000       float64
2001       float64
2002       float64
2003       float64
2004       float64
2005       float64
2006       float64
2007       float64
2008       float64
2009       float64
2010       float64
2011       float64
2012       float64
2013       float64
dtype: object


,country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Austria,388.0,397.0,412.0,425.0,438.0,452.0,463.0,475.0,487.0,501.0,511.0,519.0,492.0,498.0,501.0,504.0,508.0,511.0,514.0,522.0,530.0,537.0,542.0,546.0
1,Belgium,387.0,396.0,399.0,407.0,416.0,421.0,427.0,433.0,440.0,448.0,456.0,460.0,462.0,464.0,467.0,468.0,470.0,473.0,477.0,479.0,480.0,488.0,489.0,491.0
2,Bulgaria,152.0,158.0,166.0,178.0,188.0,197.0,205.0,209.0,220.0,233.0,245.0,265.0,279.0,298.0,317.0,333.0,233.0,277.0,317.0,337.0,353.0,368.0,385.0,402.0
3,Croatia,166.0,160.0,147.0,139.0,150.0,155.0,184.0,205.0,221.0,235.0,261.0,275.0,285.0,298.0,308.0,319.0,331.0,350.0,360.0,358.0,355.0,355.0,339.0,341.0
4,Cyprus,305.0,315.0,320.0,321.0,325.0,335.0,341.0,348.0,365.0,372.0,384.0,397.0,404.0,419.0,458.0,477.0,492.0,529.0,557.0,563.0,551.0,545.0,549.0,553.0
5,Czech Republic,234.0,247.1,260.1,274.0,283.0,295.0,310.0,329.0,339.0,335.0,336.0,346.0,358.0,364.0,374.0,387.0,401.0,414.0,424.0,424.0,429.0,436.0,448.0,450.0
6,Denmark,309.0,309.0,310.0,311.0,309.0,320.0,330.0,337.0,342.0,346.0,347.0,349.0,351.0,455.0,455.1,455.1,455.0,466.0,468.0,442.1,381.0,394.0,399.0,405.0
7,Estonia,154.0,171.0,188.0,215.0,233.0,269.0,289.0,307.0,327.0,327.0,333.0,294.0,291.0,318.0,347.0,366.0,413.0,391.0,413.0,409.0,416.0,433.0,456.0,478.0
8,Finland,388.0,382.0,383.0,369.0,367.0,372.0,379.0,378.0,392.0,403.0,412.0,416.0,422.0,436.0,448.0,462.0,475.0,485.0,507.0,519.0,535.0,551.0,560.0,570.0
9,France,404.0,406.0,408.0,413.0,420.0,422.0,427.0,435.0,446.0,454.0,460.0,467.0,471.0,475.0,476.0,476.0,478.0,480.0,483.1,486.0,538.0,540.0,542.0,546.0


In [163]:
cars.to_csv('cars_cleaned.csv', index=False)